In [1]:
print("hello")

hello


In [7]:
from langchain_community.utilities import SQLDatabase
from langchain_community.llms import Ollama
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType

In [ ]:
def setup_sql_agent(db_path: str):
    
    abs_db_path